## Saving and Restoring a Model

When using the Sequential API or the Functional API, saving a trained Keras model is as simple as it gets:

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
X_valid, X_train = X_train_full[:5000]/255.0, X_train_full[25000:]/255.0
y_valid, y_train = y_train_full[:5000], y_train_full[25000:]

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = X_train.shape[1:]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [5]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Train on 35000 samples, validate on 5000 samples
Epoch 1/30
35000/35000 [==============================] - 4s 104us/sample - loss: 0.8233 - accuracy: 0.7355 - val_loss: 0.5602 - val_accuracy: 0.8136
Epoch 2/30
35000/35000 [==============================] - 3s 94us/sample - loss: 0.5310 - accuracy: 0.8194 - val_loss: 0.4994 - val_accuracy: 0.8210
Epoch 3/30
35000/35000 [==============================] - 3s 92us/sample - loss: 0.4751 - accuracy: 0.8354 - val_loss: 0.4615 - val_accuracy: 0.8400
Epoch 4/30
35000/35000 [==============================] - 4s 111us/sample - loss: 0.4479 - accuracy: 0.8453 - val_loss: 0.4325 - val_accuracy: 0.8556
Epoch 5/30
35000/35000 [==============================] - 4s 104us/sample - loss: 0.4261 - accuracy: 0.8518 - val_loss: 0.4170 - val_accuracy: 0.8526
Epoch 6/30
35000/35000 [==============================] - 3s 96us/sample - loss: 0.4076 - accuracy: 0.8572 - val_loss: 0.4226 - val_accuracy: 0.8572
Epoch 7/30
35000/35000 [==============================

In [6]:
model.save('my_keras_model.h5')

Keras will use the HDF5 format to save both the model's arhitecture (including every layer's hyperparameters) and the values of all the model parameters for every layer (e.g., connection weights and biases). It also saves the optimizer (including its hyperparameters and any state it may have). You will typically have a script that trains a model and saves it, and one or more scripts (or web services) that load the model and use it to make predictions. Loading the model is just as easy.

In [7]:
model = keras.models.load_model('my_keras_model.h5')

This will work when using the Sequential Api or the Fucntional API, but unfortunately not when using model subclassing. You can use save_weights() and load_weights() to at least save and restore the model parameters, but you will need to save and restore everything yourself. 

But what if training lasts several hours? This is quite common, especially when training on large datasets. In this case, you should not only save your model at the end of training , but also save checkpoints at regular intervals during training, to avoid losing everything if your computer crashes. But how can you tell the fit() method to save checkpoints? Use callbacks.

## Using Callbacks

The fit() method accepts a callbacks argument that lets you specify a list of objects that Keras will call at the start and at the end of training , at the start and end of each epoch, and even before and after processing each batch. For example, the ModelCheckpoint callback saves checkpoints of your model at regular intervals during training, by default at the end of each epoch:

In [8]:
#build and compile the model
model_0 = keras.models.Sequential([
    keras.layers.Flatten(input_shape = X_train.shape[1:]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model_0.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model_0.h5")
history_0 = model_0.fit(X_train, y_train, epochs=10, callbacks=[checkpoint_cb])

Train on 35000 samples
Epoch 1/10
35000/35000 [==============================] - 4s 122us/sample - loss: 0.8165 - accuracy: 0.7365
Epoch 2/10
35000/35000 [==============================] - 3s 97us/sample - loss: 0.5304 - accuracy: 0.8193
Epoch 3/10
35000/35000 [==============================] - 4s 104us/sample - loss: 0.4813 - accuracy: 0.8333
Epoch 4/10
35000/35000 [==============================] - 4s 119us/sample - loss: 0.4521 - accuracy: 0.8427
Epoch 5/10
35000/35000 [==============================] - 3s 95us/sample - loss: 0.4300 - accuracy: 0.8522
Epoch 6/10
35000/35000 [==============================] - 3s 81us/sample - loss: 0.4119 - accuracy: 0.8570s - loss: 0.4109 - accu
Epoch 7/10
35000/35000 [==============================] - 3s 85us/sample - loss: 0.3967 - accuracy: 0.8618
Epoch 8/10
35000/35000 [==============================] - 3s 83us/sample - loss: 0.3854 - accuracy: 0.8647
Epoch 9/10
35000/35000 [==============================] - 3s 88us/sample - loss: 0.3727 - accur

Morove if you use a validation set during training you can use save_best_only = True when creating the ModelCheckpoint. In this case, it will only save your model when it's performance on the validation set is the best so far. This way, you do not need to worry about training for too long and overfitting the training set: simply restore the last model saved after training, and this will be the best model on the validation set. The following code is simple way to implement early stopping:


In [9]:
model_1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape = X_train.shape[1:]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model_1.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('my_keras_model_1.h5', save_best_only=True)
history_1 = model_1.fit(X_train, y_train,epochs=10,validation_data=(X_valid, y_valid),callbacks=[checkpoint_cb])
model = keras.models.load_model('my_keras_model_1.h5') #roll back to best model

Train on 35000 samples, validate on 5000 samples
Epoch 1/10
35000/35000 [==============================] - 3s 100us/sample - loss: 0.8160 - accuracy: 0.7424 - val_loss: 0.6027 - val_accuracy: 0.7934
Epoch 2/10
35000/35000 [==============================] - 3s 89us/sample - loss: 0.5281 - accuracy: 0.8212 - val_loss: 0.4923 - val_accuracy: 0.8338
Epoch 3/10
35000/35000 [==============================] - 3s 89us/sample - loss: 0.4773 - accuracy: 0.8339 - val_loss: 0.4476 - val_accuracy: 0.8466
Epoch 4/10
35000/35000 [==============================] - 3s 91us/sample - loss: 0.4461 - accuracy: 0.8454 - val_loss: 0.4547 - val_accuracy: 0.8416
Epoch 5/10
35000/35000 [==============================] - 3s 90us/sample - loss: 0.4271 - accuracy: 0.8514 - val_loss: 0.4138 - val_accuracy: 0.8564
Epoch 6/10
35000/35000 [==============================] - 3s 88us/sample - loss: 0.4085 - accuracy: 0.8576 - val_loss: 0.4165 - val_accuracy: 0.8592
Epoch 7/10
35000/35000 [==============================] 

Another way to implement early stopping is to simply use the EarlyStopping callback. It will interrupt training when it measures no progress on the validation set for a number of epoch (defined by the patience argument), and it will optionally roll back to the best model. You can combine both callbacks to checkpoints of your model (in case your computer crashes) and interrupt training early when there is no more progress (to avoid wasting time and resources):

In [10]:
model_2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape = X_train.shape[1:]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model_2.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
checkpoint_cb_2 = keras.callbacks.ModelCheckpoint('my_keras_model_2.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history_2 = model_2.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
                        callbacks=[checkpoint_cb_2, early_stopping_cb])

Train on 35000 samples, validate on 5000 samples
Epoch 1/100
35000/35000 [==============================] - 4s 106us/sample - loss: 0.7840 - accuracy: 0.7497 - val_loss: 0.5675 - val_accuracy: 0.8082
Epoch 2/100
35000/35000 [==============================] - 4s 108us/sample - loss: 0.5192 - accuracy: 0.8233 - val_loss: 0.5012 - val_accuracy: 0.8292
Epoch 3/100
35000/35000 [==============================] - 4s 104us/sample - loss: 0.4695 - accuracy: 0.8362 - val_loss: 0.4539 - val_accuracy: 0.8434
Epoch 4/100
35000/35000 [==============================] - 4s 101us/sample - loss: 0.4400 - accuracy: 0.8459 - val_loss: 0.4248 - val_accuracy: 0.8580
Epoch 5/100
35000/35000 [==============================] - 3s 99us/sample - loss: 0.4185 - accuracy: 0.8551 - val_loss: 0.4213 - val_accuracy: 0.8570
Epoch 6/100
35000/35000 [==============================] - 4s 101us/sample - loss: 0.4025 - accuracy: 0.8597 - val_loss: 0.3914 - val_accuracy: 0.8694
Epoch 7/100
35000/35000 [=====================

The number of epochs can be set to a large value since training will stop automatically when there is no more progress. In this case, there is no need to restore the best model saved because the EarlyStopping callback will keep track of the best weights and restore them for you at the end of training.

There is many other callbacks available in the keras.callbacks package (https://keras.io/callbacks/.)

If you need extra control, you can easily write your own custom callbacks. As an example of how to do that, 
the following custom callback will display the ratio between the validation loss and training loss during training(e.g. to detect overfitting):

In [11]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print('\nval/train: {:2f}'.format(logs['val_loss']/logs['loss']))

As you might expect you can implement on_train_begin(), on_train_end(), on_epoch_begin(), on_epoch_end(),on_batc_begin(),on_natch_end(). Callbacks can also be used during evaluation and predictions, should you ever need them on_test_end(), on_test_batch_begin(), or on_test_batch_end() (called by evaluate()), and for prediction you should implement on_predict_begin(), on_predict_end(), on_predict_batch_begin(), or on_predict_batch_end() (called by predict()).

Now let's take a look at one more tool from tf.keras: TensorBoard

## Using TensorBoard for Visualization